# Decepticons: Corrupted Transformers Breach Privacy in Federated Learning for Language Models

This notebook shows an example for the threat model and attack described in "Decepticons: Corrupted Transformers Breach Privacy in Federated Learning for Language Models
". This example deviates from the other "honest-but-curious" server models and investigates a malicious server that may send malicious server updates. The attack succeeds for a range of common transformer architectures and works merely by sending a single malicious query to the user model.

In this notebook, we attack the tiny transformer model discussed in "Advances and Open Problems in Federated Learning" (https://arxiv.org/abs/1912.04977). The model architecture is unchanged.



Paper URL: https://arxiv.org/abs/2201.12675

### Abstract:
A central tenet of Federated learning (FL), which trains models without centralizing user data, is privacy. However, previous work has shown that the gradient updates used in FL can leak user information. While the most industrial uses of FL are for text applications (e.g. keystroke prediction), nearly all attacks on FL privacy have focused on simple image classifiers. We propose a novel attack that reveals private user text by deploying malicious parameter vectors, and which succeeds even with mini-batches, multiple users, and long sequences. Unlike previous attacks on FL, the attack exploits characteristics of both the Transformer architecture and the token embedding, separately extracting tokens and positional embeddings to retrieve high-fidelity text. This work suggests that FL on text, which has historically been resistant to privacy attacks, is far more vulnerable than previously thought.

### Startup

In [1]:
try:
    import breaching
except ModuleNotFoundError:
    # You only really need this safety net if you want to run these notebooks directly in the examples directory
    # Don't worry about this if you installed the package or moved the notebook to the main directory.
    import os; os.chdir("..")
    import breaching
    
import torch
%load_ext autoreload
%autoreload 2

# Redirects logs directly into the jupyter notebook
import logging, sys
logging.basicConfig(level=logging.INFO, handlers=[logging.StreamHandler(sys.stdout)], format='%(message)s')
logger = logging.getLogger()

### Initialize cfg object and system setup:

This will load the full configuration object. This includes the configuration for the use case and threat model as `cfg.case` and the hyperparameters and implementation of the attack as `cfg.attack`. All parameters can be modified below, or overriden with `overrides=` as if they were cmd-line arguments.

In [2]:
cfg = breaching.get_config(overrides=["attack=decepticon", "case=10_causal_lang_training", 
                                     "case/server=malicious-transformer"])
          
device = torch.device('cpu')
torch.backends.cudnn.benchmark = cfg.case.impl.benchmark
setup = dict(device=device, dtype=getattr(torch, cfg.case.impl.dtype))
setup

Investigating use case causal_lang_training with server type malicious_transformer_parameters.


{'device': device(type='cpu'), 'dtype': torch.float32}

### Modify config options here

You can use `.attribute` access to modify any of these configurations for the attack, or the case:

In [10]:
cfg.case.user.num_data_points = 1 # How many sentences?
cfg.case.user.user_idx = 1 # From which user?
cfg.case.data.shape = [32] # This is the sequence length

cfg.case.server.has_external_data = True  # Not strictly necessary, could also use random text (see Appendix)
cfg.case.data.tokenizer = "word-level"

# Attack hyperparameters:

# this option requires installation of `k-means-constrained` which can be tricky:
# If this doesn't work for you, falling back to "dynamic-threshold" is still a decent option.
cfg.attack.sentence_algorithm = "k-means" 

cfg.attack.undivided=True

cfg.case.user.local_diff_privacy.input_noise = 0.0  # input up to 1.0 is ok
cfg.case.user.local_diff_privacy.gradient_noise =1e-6
cfg.case.user.local_diff_privacy.per_example_clipping = 0
cfg.case.user.local_diff_privacy.distribution = 'gaussian'

cfg.attack.breach_reduction = "total-weight"


# Server side:
cfg.case.server.param_modification.v_length = 8 # Length of the sentence component
cfg.case.server.param_modification.eps = 1
cfg.case.server.param_modification.measurement_scale=1  # 1e12 # Circumvent GELU
cfg.case.server.param_modification.imprint_sentence_position = 0
cfg.case.server.param_modification.softmax_skew = 1e8
cfg.case.server.param_modification.sequence_token_weight = 1

### Instantiate all parties

The following lines generate "server, "user" and "attacker" objects and print an overview of their configurations.

In [11]:
user, server, model, loss_fn = breaching.cases.construct_case(cfg.case, setup)
attacker = breaching.attacks.prepare_attack(server.model, server.loss, cfg.attack, setup)
breaching.utils.overview(server, user, attacker)

Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Model architecture transformer3 loaded with 10,800,433 parameters and 0 buffers.
Overall this is a data ratio of  337514:1 for target shape [1, 32] given that num_queries=1.
User (of type UserSingleStep) with settings:
    Number of data points: 1

    Threat model:
    User provides labels: False
    User provides buffers: False
    User provides number of data points: True

    Data:
    Dataset: wikitext
    user: 1
    Defense: Local gaussian gradient noise with strength 9.999999974752427e-07.
        
Server (of type MaliciousTransformerServer) with settings:
    Threat model: Malicious (Parameters)
    Number of planned queries: 1
    Has external/public data: True

    Model:
        model specification:

### Simulate an attacked FL protocol

This exchange is a simulation of a single query in a federated learning protocol. The server sends out a `server_payload` and the user computes an update based on their private local data. This user update is `shared_data` and contains, for example, the parameter gradient of the model in the simplest case. `true_user_data` is also returned by `.compute_local_updates`, but of course not forwarded to the server or attacker and only used for (our) analysis.

In [12]:
server_payload = server.distribute_payload()
shared_data, true_user_data = user.compute_local_updates(server_payload)  

Found attention of shape torch.Size([288, 96]).
Computing feature distribution before the probe layer Linear(in_features=96, out_features=1536, bias=True) from external data.
Feature mean is 0.9348141551017761, feature std is 9.500455856323242.
Computing user update on user 1 in model mode: eval.


In [13]:
user.print(true_user_data)

[CLS] the tower building of the little rock arsenal, also known as u. s. arsenal building, is a building located in macarthur park in downtown little rock,


### Reconstruct user data:

Now we launch the attack, reconstructing user data based on only the `server_payload` and the `shared_data`. 

For this attack, we also share secret information from the malicious server with the attack (`server.secrets`), which here is the location and structure of the imprint block.

In [14]:
reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], server.secrets, 
                                                      dryrun=cfg.dryrun)

Recovered tokens tensor([[    1,     5,     5,     6,     6,     7,     7,     7,    10,    11,
            12,    12,    16,    29,    32,    72,   494,   566,   566,   845,
           845,   942,   992,  1085,  1085,  1085,  1936,  1936,  5503,  6073,
          8102, 18571]]) through strategy decoder-bias.
Recovered 4608 embeddings with positional data from imprinted layer.
tensor([5.4315e-07, 4.5697e-07, 4.4792e-07, 4.4228e-07, 4.3629e-07, 4.1989e-07,
        4.1529e-07, 4.0584e-07, 4.0467e-07, 4.0372e-07, 4.0360e-07, 4.0319e-07,
        3.9966e-07, 3.9831e-07, 3.8934e-07, 3.8635e-07, 3.8568e-07, 3.8438e-07,
        3.8066e-07, 3.7897e-07, 3.7632e-07, 3.7599e-07, 3.7199e-07, 3.6933e-07,
        3.6793e-07, 3.6771e-07, 3.6459e-07, 3.6317e-07, 3.6261e-07, 3.6077e-07,
        3.5836e-07, 3.5832e-07, 3.5739e-07, 3.5383e-07, 3.5255e-07, 3.4910e-07,
        3.4875e-07, 3.4853e-07, 3.4683e-07, 3.4595e-07, 3.4536e-07, 3.4397e-07,
        3.4306e-07, 3.4291e-07, 3.4172e-07, 3.3504e-07, 3.325

Next we'll evaluate metrics, comparing the `reconstructed_user_data` to the `true_user_data`.

In [15]:
metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, order_batch=True, compute_full_iip=False, 
                                    cfg_case=cfg.case, setup=setup)

METRICS: | Accuracy: 0.3750 | S-BLEU: 0.14 | FMSE: 2.1147e-01 | 
 G-BLEU: 0.34 | ROUGE1: 1.00| ROUGE2: 0.19 | ROUGE-L: 0.44| Token Acc: 100.00% | Label Acc: 100.00%


And finally, we also plot the reconstructed data:

In [9]:
user.print(reconstructed_user_data)

located in known, arsenal as the [CLS] little building tower u is. in s a. rock downtown of macarthur also rock little building building arsenal, the park,


### Notes:
* There are a variety of hyperparameters to the attack which are set to reasonable defaults. Performance of the attack could be improved in some unusual use cases (datasets or models) by tuning these parameters further.
* In this example, dropout is disabled under the assumption that this is a parameter that can be controlled in the server update. The optimal attack simply disables dropout. However, the attack can still succeed when dropout is enforced by the user, albeit with a minor loss in reconstruction quality.
* Try increasing `num_data_points` or `data.shape`